In [1]:
import pandas as pd
import altair as alt
import os

In [2]:
os.chdir('/Users/sambickel-barlow/Desktop/ExEcon_charts code')

In [4]:
chil_ex_23 = pd.read_csv('data/Santiago/product-exports-1.csv')
chil_ex_18 = pd.read_csv('data/Santiago/product-exports-1 2.csv')
chil_ex_19 = pd.read_csv('data/Santiago/product-exports-1 3.csv')
chil_ex_20 = pd.read_csv('data/Santiago/product-exports-1 4.csv')
chil_ex_21 = pd.read_csv('data/Santiago/product-exports-1 5.csv')
chil_ex_22 = pd.read_csv('data/Santiago/product-exports-1 6.csv')

In [5]:
chil_ex = chil_ex_18.merge(chil_ex_19, how='outer', on=['HS4','HS4 ID'])
chil_ex.rename(columns={'Current Exports Value_x': '2018'}, inplace=True)
chil_ex.rename(columns={'Current Exports Value_y': '2019'}, inplace=True)
chil_ex = chil_ex.merge(chil_ex_20, how='outer', on=['HS4','HS4 ID'])
chil_ex.rename(columns={'Current Exports Value': '2020'}, inplace=True)
chil_ex = chil_ex.merge(chil_ex_21, how='outer', on=['HS4','HS4 ID'])
chil_ex.rename(columns={'Current Exports Value': '2021'}, inplace=True)
chil_ex = chil_ex.merge(chil_ex_22, how='outer', on=['HS4','HS4 ID'])
chil_ex.rename(columns={'Current Exports Value': '2022'}, inplace=True)
chil_ex = chil_ex.merge(chil_ex_23, how='outer', on=['HS4','HS4 ID'])
chil_ex.rename(columns={'Current Exports Value': '2023'}, inplace=True)

In [6]:
chil_ex = chil_ex.melt(id_vars=['HS4', 'HS4 ID'], var_name='Year', value_name='Export Value')

In [7]:
# Consolidate HS4s outside the top 49 in 2023 into 'Other' for ex_chart
top_49_hs4 = (chil_ex[chil_ex['Year'] == '2023']
                .sort_values('Export Value', ascending=False)
                .head(832)['HS4'])
chil_ex_grouped = chil_ex.copy()
chil_ex_grouped['HS4_grouped'] = chil_ex_grouped['HS4'].where(chil_ex_grouped['HS4'].isin(top_49_hs4), 'Other')

In [8]:
# Aggregate Export Value by HS4_grouped and Year
chil_ex_grouped_sum = chil_ex_grouped.groupby(['HS4_grouped', 'Year'], as_index=False)['Export Value'].sum()

In [9]:
# Calculate percent of total export value for each HS4_grouped in each year
chil_ex_grouped_sum['Year_total'] = chil_ex_grouped_sum.groupby('Year')['Export Value'].transform('sum')
chil_ex_grouped_sum['Export Percent'] = chil_ex_grouped_sum['Export Value'] / chil_ex_grouped_sum['Year_total']

In [34]:
# Define the top 4 copper products explicitly
top_copper_products = ['Copper Ore', 'Raw Copper', 'Refined Copper', 'Copper Wire']
# Find all HS4s containing 'Copper'
copper_hs4s = chil_ex['HS4'][chil_ex['HS4'].str.contains('Copper')].unique().tolist()
# Assign 'Other Copper Products' to copper HS4s not in the top 4
def copper_legend_label(x):
    if x in top_copper_products:
        return x
    elif x in copper_hs4s:
        return 'Other Copper Products'
    else:
        return 'All Other exports'
# Update color mapping: all copper products get the same copper orange color
copper_orange = '#FFA500'  # Main copper orange color
blue_color = '#002fa7'  # Nice blue from Aceh chart
product_color_map = {
    top_copper_products[0]: copper_orange,
    top_copper_products[1]: copper_orange,
    top_copper_products[2]: copper_orange,
    top_copper_products[3]: copper_orange,
    'Other Copper Products': copper_orange,
    'All Other exports': blue_color
}
# Create a legend group: assign labels using the function above
chil_ex_grouped_sum['HS4_legend'] = chil_ex_grouped_sum['HS4_grouped'].apply(copper_legend_label)
# Ensure Export Percent is float for Altair and scale to 0-100 for percent axis
chil_ex_grouped_sum['Export Percent'] = chil_ex_grouped_sum['Export Percent'].astype(float)
# Chart for all other lines (no legend) - updated with Aceh aesthetics and blue color with opacity
other_chart = alt.Chart(chil_ex_grouped_sum).mark_line(interpolate='monotone', opacity=0.5).encode(
    x=alt.X('Year', scale=alt.Scale(padding=0), axis=alt.Axis(title=None, labelAngle=0, domainColor='dimgrey', labelColor='#333333', tickColor='#333333', labelFontWeight='bold', labelFontSize=12)),
    y=alt.Y('Export Percent:Q', title='Export Share', axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, titleX=-50, titleColor='#333333', format='.0%', grid=False, domainColor='dimgrey', labelColor='#333333', tickColor='#333333', labelFontWeight='bold', labelFontSize=12)),
    color=alt.value(blue_color),
    detail='HS4_grouped:N',
    tooltip=[
        alt.Tooltip('HS4_grouped:N', title='Product Category'),
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Export Percent:Q', title='Share of Total Exports', format='.1%')
    ]
 ).transform_filter(
    (alt.datum.HS4_legend == 'All Other exports') & (alt.datum.HS4_grouped != 'All Other exports')
)
# Chart for copper products and 'All Other exports' (for legend)
main_chart = alt.Chart(chil_ex_grouped_sum).mark_line(interpolate='monotone').encode(
    x=alt.X('Year', scale=alt.Scale(padding=0)),
    y=alt.Y('Export Percent:Q'),
    color=alt.Color('HS4_legend', scale=alt.Scale(domain=list(product_color_map.keys()), range=list(product_color_map.values())), legend=None),
    tooltip=[
        alt.Tooltip('HS4_grouped:N', title='Product Category'), 
        alt.Tooltip('Year:O', title='Year'), 
        alt.Tooltip('Export Percent:Q', title='Share of Total Exports', format='.1%')
    ]
 ).transform_filter(
    (alt.datum.HS4_legend != 'All Other exports') | (alt.datum.HS4_grouped == 'All Other exports')
)

# Get the last points for each legend category in 2023
last_points = chil_ex_grouped_sum[chil_ex_grouped_sum['Year'] == '2023'].copy()

# For "All Other exports", we need to select just one representative point
# Let's use the one with the highest export percentage to avoid overlapping labels
other_exports_last = last_points[last_points['HS4_legend'] == 'All Other exports']
if not other_exports_last.empty:
    # Keep only the highest "All Other exports" point for labeling
    other_exports_max = other_exports_last.loc[other_exports_last['Export Percent'].idxmax()]
    # Remove all "All Other exports" entries and add back just the max one
    last_points = last_points[last_points['HS4_legend'] != 'All Other exports']
    last_points = pd.concat([last_points, other_exports_max.to_frame().T])

# Sort by Export Percent to help with positioning
last_points = last_points.sort_values('Export Percent', ascending=False).reset_index(drop=True)

# Add manual vertical adjustments (dy) to prevent overlapping labels
# These adjustments are in pixels - negative moves up, positive moves down
label_adjustments = {
    'Copper Ore': 0,           # Highest line, keep at actual position
    'Raw Copper': -2,           # Second highest, keep at actual position  
    'Refined Copper': 0,      # Move up slightly to avoid overlap
    'Copper Wire': -5,          # Move down slightly
    'Other Copper Products': 3, # Keep at actual position
    'All Other exports': 10    # Move up slightly to avoid any overlap
}

# Apply the adjustments
last_points['dy_adjust'] = last_points['HS4_legend'].map(label_adjustments).fillna(0)

# Create individual text label charts for each category to apply different dy values
text_charts = []

for _, row in last_points.iterrows():
    chart = alt.Chart(pd.DataFrame([row])).mark_text(
        align='left',
        dx=5,
        dy=row['dy_adjust'],  # Apply individual vertical adjustment
        fontWeight='bold',
        fontSize=12
    ).encode(
        x=alt.X('Year'),
        y=alt.Y('Export Percent:Q'),
        text=alt.Text('HS4_legend:N'),
        color=alt.value(product_color_map[row['HS4_legend']])  # Direct color value
    )
    text_charts.append(chart)

# Combine all text charts
text_labels = alt.layer(*text_charts)
# Custom Altair theme with Aceh aesthetics - light background and blue-grey colors
custom_config = alt.Config(
    background='#fbfbfb',  # Light background like Aceh style
    view=alt.ViewConfig(stroke='transparent'),
    axis=alt.AxisConfig(
        labelColor='#333333',  # Dark grey like Aceh style
        domainColor='dimgrey',
        tickColor='#333333',   # Dark grey like Aceh style
        titleColor='#333333',  # Dark grey like Aceh style
        labelFontWeight='bold',  # Bold font weight like Aceh style
        labelFontSize=12      # Font size like Aceh style
    ),
    legend=alt.LegendConfig(
        labelColor='#333333',  # Dark grey like Aceh style
        titleColor='#333333',  # Dark grey like Aceh style
        labelFontWeight='bold',
        labelFontSize=12
    ),
    title=alt.TitleConfig(
        anchor='start',
        color='#333333',  # Dark grey like Aceh style
        fontSize=15,      # Font size like Aceh style
        fontWeight='normal',
        font='Newcastle Basic Clean, Arial, sans-serif'  # Font like Aceh style
    ),
    font='Newcastle Basic Clean, Arial, sans-serif'  # Global font like Aceh style
)
# Overlay all charts (other lines, main lines, and text labels)
ex_chart = (other_chart + main_chart + text_labels).configure(**custom_config.to_dict()).properties(
    title=alt.TitleParams(
        text='Chilean exports',
        subtitle=['Export composition by product category, 2018-2023', ''],
        fontSize=16,
        subtitleFontSize=13,
        subtitleFontStyle='italic',
        subtitleColor='#333333',
        color='#333333',
        fontWeight='bold',  # Make title bold
        anchor='start',
        font='Newcastle Basic Clean, Arial, sans-serif'
    ), 
    width=600, 
    height=300
)
ex_chart

alt.LayerChart(...)

In [35]:
# Save to png
ex_chart.save('Santiago.png', scale_factor=2)
ex_chart.save('Santiago.json', scale_factor=2)